In [4]:
import torch
from torch import nn

In [2]:
import numpy as np

In [3]:
import librosa

AttributeError: module 'torch.nn' has no attribute 'AveragePooling2D'

In [35]:
class VoxNet(torch.nn.Module):
    def __init__(self, n_classes=1000):
        
        super(VoxNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=96, kernel_size=(7, 7), 
                      stride=(2, 2)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2)),
            
            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=(5, 5), 
                      stride=(2, 2)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2)),
            
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3, 3), 
                      stride=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3, 3), 
                      stride=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3, 3), 
                      stride=(1, 1)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(5, 3), stride=(3, 2)),
        )
        
        self.fc6_b = nn.Linear(256, 4096)
        self.fc7 = nn.Linear(4096, 4096)
        self.fc8 = nn.Linear(4096, n_classes)
    
    def forward(self, x):
        conv = self.conv(x)
        # print(conv.size())
        flattened = torch.nn.functional.avg_pool2d(conv, kernel_size=(7, 5), stride=1)
        flattened = flattened.view(flattened.size(0), 256)
        # print(flattened.size())
        flattened = self.fc6_b(flattened)
        flattened = self.fc7(flattened)
        flattened = self.fc8(flattened)
        return flattened

In [36]:
vox = VoxNet(n_classes=50)

In [37]:
vox

VoxNet(
  (conv): Sequential(
    (0): Conv2d(1, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=(5, 3), stride=(3, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (fc6_b): Linear(in_features=256, out_features=4096, bias=True)
  (fc7): Linear(in_features=4096, out_features=4096, bias=True)
  (fc8): Linear(in_features=4096, out_features=50, bias=True)
)

In [41]:
batch_size = 4

In [42]:
with torch.no_grad():
    x = torch.randn(batch_size, 1, 512, 300)

In [43]:
o = vox(x)

In [44]:
o.size()

torch.Size([4, 50])